# **Основные понятия и определения в области NoSQL-решений, OLAP-технологий**

- Ограничения CAP теоремы
- Понятие NoSQL, классификация NoSQL-решений
- Распределенные хранилища данных. Технологии шардинга, репликации, MapReduce
- OLAP-технологии

## Ограничения CAP теоремы

Несмотря на широкое использованием реляционной модели данных, практика ее повсеместного внедрения позволила выявить и ее недостатки. При этом ее основные достоинства, как ни странно, оказываются и ее важными ограничениями и недостатками при использовании реляционной модели в ряде предметных областей, набирающих в последнее время существенную популярность. В первую очередь это касается области больших данных с требованием их распределенного хранения. Также особые сложности возникают при сопровождении реляционной базы данных в течение длительного времени с требованием сохранить хронологию изменений, или требованиями изменения самой структуры реляционной базы данных. 

Рассмотрим возникающие ограничения по использованию реляционной модели более подробно. 
Термин Big Data появился в конце 2000 годов и одна из его составляющих означает массив структурированных и неструктурированных данных значительного объема (принципиально не может поместиться за раз не только в оперативной памяти отдельного компьютера, но и на винчестере одного компьютера). В этой ситуации приходится каким-то способом дробить данные на части, хранить их распределенно, синхронизировать их при выдаче. Часто при этом данные подвержены очень быстрым изменениям. Все это накладывает определенные требования на информационные системы, служащие для обработки таких данных. 

Обычно к информационной системе в отношении обработки данных предъявляют следующий состав требований:

- **Согласованность (consistency - С) данных**  - означает, что данные не противоречат друг другу;
- **Доступность (availability - A)** – каждый запрос к информационной системе дает ответ без существенной задержки по времени;
- **Масштабируемость (partition - P)** – имеется принципиальная возможность сегментации данных для хранения их на удаленных сегментах распределенной информационной системы.

Возможность совместного удовлетворения этих требований показано на рисунке. На данном рисунке проведена аналогия с тремя другими требованиями к разработке любой сложной технической системе: качество, время, стоимость. Практика построения таких систем показывает, что, как правило, всегда удается добиться только любых двух из этих трех требований.

![ris1.png](ris1.png)

В 2000 году Эрик Брюер сформулировал аналогичное эмпирическое утверждение относительно совместного достижения требований C, A, P к возможностям хранения и обработки данных в информационных системах. Данное утверждение известно как 

**CAP-теорема: на практике всегда можно добиться одновременного выполнения только любых двух из этих трех требований: C - согласованность, A - доступность, P – масштабируемость.**

В дальнейшем некоторые ученые подвергали это утверждение критике за его недостаточно строгую формулировку.
Однако в 2002 году ученые из MIT подобрали формальные модели асинхронных и синхронных распределённых вычислений, в которых CAP теорема выполняется. После чего многие авторы ссылаются на CAP гипотезу как на доказанную теорему.

Таким образом, в соответствии с CAP теоремой возможно построение трех классов систем: CA системы, AP системы, CP системы.

Как видно из рисунка, информационные системы, построенные с использованием реляционной модели данных (MySQL, MariaDB, MS SQL Server и др.), относятся именно к CA системам, а значит на пути их масштабирования всегда будут возникать сложности.

В этой связи, описывая два других класса систем CP, AP, принято использовать обозначение NoSQL, указывающего на отказ в той или иной степени от реляционной модели данных. Концепция NoSQL использует CAP теорему, как обоснование отказа от попыток добиться полной согласованности данных, или их доступности при их распределенном хранении. 

Следующий вид ограничений, которые сопровождают реляционную модель связан с ее структурой, учитывающей требования нормализации данных.

В таблицах реляционной модели целесообразно исключать атрибуты, в которых большинство записей имеют значение NULL, обозначающее отсутствие значения. Если мы хотим указать, что один из ста служащих имеет особую квалификацию, для хранения этой информации не следует добавлять в таблицу еще один столбец, поскольку для остальных 99 работников значением столбца будет NULL. Вместо этого следует добавить новую таблицу, в которой будут храниться только кодовые номера и информация о квалификации тех работников, которых это касается. Решение перечисленных проблем состоит в разделении данных и связей, что и обеспечивается процедурой нормализации. Однако в условиях длительной эксплуатации базы данных такие ситуации неизбежно возникают, что приводит к необходимости добавления новых таблиц. Их количество со временем может увеличиться до неприемлемого значения. 

Но еще большая проблема связана с фиксированной жёсткой структурой базы данных. Дело в том, что в процессе эксплуатации базы данных меняются не только значения атрибутов, но и их состав, и, самое главное, тип связей между сущностями. Добавление новых атрибутов в таблицу требует возобновить анализ на выявление зависимостей этого атрибута с другими атрибутами, требует, как правило, переработки программных приложений, обслуживающих базу данных. Это приводит к потере соответствия между моделью представления данных в предметной области и моделью в реляционном подходе. 

Еще более затратно изменение структуры базы данных. Допустим, две таблицы были связаны в отношении один ко многим, однако в дальнейшем отношение поменялось на многие ко многим (например, в базе данных учебного отдела таблица группа связана с таблицей студенты в отношении один ко многим, однако в дальнейшем выяснилось, что один и тот же студент может учиться в разных группах по разным направлениям и формам обучения). Обычно это решается добавлением новой таблицы, с которой две указанные связываются отношением один ко многим, а также существенной переработкой программных приложений, обслуживающих базу данных. Но при этом изменение структуры происходит без возможности ее отката назад. Иными словами, хронология структуры базы данных в реляционной модели никак не учитывается. Она в каждый момент времени остается жесткой и фиксированной.

Также в настоящее время к данным из БД применяются различные методы многомерного анализа данных, которые приводят к использованию методов Data Mining, OLAP-технологий. При этом многие исследователи показываю, что при проведении OLAP анализа часто требуется расширять и дополнять данные, делать их избыточными, что может приводить к потере согласованности.

## Понятие NoSQL, классификация NoSQL-решений

За многолетнюю историю развития СУБД было выработано несколько моделей представления структурированной информации: иерархическая, сетевая, реляционная, объектно-реляционная и т. д. Наибольшее распространение получила реляционная модель, предложенная Эдгаром Коддом в 1970 году, которая надолго стала стандартом представления структурированной информации. 

Причины этого вытекают из сформулированных требований к СУБД. Во-первых, реляционная модель оказалась достаточно простой, с точки зрения прикладного программиста. Во-вторых, она обладает достаточной гибкостью и позволяет представлять информацию из самых разных предметных областей. В-третьих, в рамках этой модели используется мощный и удобный подход к манипулированию данными (реляционная алгебра), впоследствии оформленный в виде языка SQL. В-четвертых, простота и естественность реляционной алгебры позволили создать высокопроизводительные и универсальные алгоритмы выполнения запросов, удовлетворяющие большую часть потребностей разработчиков прикладных систем. Наконец, к сегодняшнему дню создано множество инструментальных средств, ориентированных на обработку реляционных данных, что дает еще одно преимущество реляционных СУБД.
Итак, при разработке прикладного ПО из исходных требований следует выбор модели представления информации, а уже из нее следует выбор конкретной СУБД. Эта схема может показаться абстрактной в силу того, что на текущий момент доминирующей является реляционная модель, а выбор СУБД производится из достаточно ограниченного списка, который заранее продиктован другими факторами, такими как интеграция, простота поддержки, стоимость и т. д.

Реляционная модель предполагает оперирование только атомарными данными и исключает обработку какой-либо неструктурированной информации, а это приводит к тому, что хранение графических, аудио- или видеоданных становится невозможным. Более того, невозможно даже хранение текстовых документов произвольной длины. Поэтому, первым отступлением от классической реляционной модели в сторону удобства разработки прикладного ПО можно считать введение типа BLOB (Binary Large Object — «бинарные данные большого объема»), который сегодня поддерживается большинством современных СУБД и закреплен в стандартах языка SQL. Каждая СУБД, помимо BLOB, может иметь свои собственные «дополнительные» типы данных, которые обычно требуют введения дополнительных поисковых операций (например, полнотекстовый поиск по BLOB). Это еще дальше уводит от классической реляционной модели.

По мере роста объемов хранимой информации и сложности ее внутренних взаимосвязей стали возникать новые проблемы. Увеличение сложности SQL-запросов привело к тому, что СУБД уже далеко не всегда способны выработать оптимальный план выполнения поступившего запроса. На практике это производится двумя способами. Первый — в тексте SQL указываются «подсказки», помогающие оптимизатору запросов выработать наиболее эффективный план. Второй — вместо запроса используется хранимая процедура. Причина хорошей применимости обоих способов кроется в том, что встроенные в СУБД оптимизаторы запросов располагают только статистической информацией о содержимом базы и строят планы выполнения запросов только на ее основе, а разработчик прикладного ПО всегда обладает гораздо большими сведениями.

Оба отступления от базовых принципов реляционной модели (введение неатомарных типов данных и тонкое управление алгоритмами обработки) напрямую следуют из особенностей самой модели, провозглашающей универсальность способов хранения информации и алгоритмов ее обработки. За этой универсальностью скрывается невозможность учета специфики данных, позволяющей существенно оптимизировать работу алгоритмов.

Допустим, требуется организовать хранение большого множества чисел и быстро выполнять операции поиска, добавления и удаления числа. Для этого в реляционной базе создается таблица с одним числовым полем, а по этому полю формируется поисковый индекс на основе B-дерева. Такое решение работает быстро, но предполагает дублирование информации (числа хранятся и в самой таблице, и в поисковом индексе). Более эффективным было бы использование только B-дерева, но реляционные СУБД не всегда способны это определить.

Данный пример может показаться слишком простым, но похожая ситуация реализована в Triple Storages (хранение графов семантических сетей в семантическом Web), отвечающих за хранение и обработку графов семантических связей. Каждая дуга графа представляет собой три числа (два кода вершин и один код окраски). Множество всех дуг можно было бы хранить в виде реляционной таблицы с необходимыми поисковыми индексами, но вместо этого хранят только поисковые индексы, поскольку каждый из них содержит всю информацию о каждой дуге.

Третий пример — полнотекстовые поисковые системы Интернета. Их база данных должна содержать поисковые слова и коды документов, в которых эти слова являются ключевыми. Это несложно сделать с помощью реляционной таблицы, состоящей из двух полей — поисковое слово и код документа. Если слово встречается в нескольких документах или документ имеет несколько ключевых слов, то таблица содержит несколько записей.
Возможна существенная оптимизация рассмотренного примера: скажем, вместо текстового представления слова можно хранить его целочисленный код, а вместо кода документа хранить BLOB, где будут содержаться коды всех документов, имеющих отношение к данному слову. Дальнейшая оптимизация в рамках реляционного подхода невозможна, но ее можно провести посредством так называемого инвертированного индекса. Коды документов сортируются по возрастанию, и в полученном списке вычисляются разницы между кодами соседних документов. Получается последовательность, состоящая из кода первого документа и множества чисел небольшой величины. Она хорошо сжимается алгоритмами энтропийной компрессии (например, алгоритмом Лемпеля — Зива-Велча), в результате чего значительно сокращается объем ввода/вывода, ускоряется загрузка информации с диска и в конечном счете растет скорость полнотекстового поиска.

Термин **NoSQL** стал использоваться в конце 90-х годов, но реальный смысл в том виде, как он используется сейчас, приобрел только в середине 2009 года. Изначально так называлась открытая база данных, созданная Карло Строззи, которая хранила все данные как текстовые файлы и использовала shell скрипты вместо SQL для доступа к данным. 

Таким образом, термин NoSQL имеет стихийное происхождение и скорее характеризует вектор развития IT-сферы в сторону от реляционных баз данных. Расшифровывается как Not Only SQL, хотя есть сторонники и прямого определения No SQL.

NoSQL стал общим термином для различных баз данных и хранилищ, но он не обозначает какую-либо одну конкретную технологию или продукт.

Выделим основные тренды в формировании представлений о NoSQL.

1. Первый тренд — увеличение объемов хранимых данных. Сегодня хранилища достигли таких невероятных размеров, nолько за 2009 и 2010 годы в базах было сохранено больше информации, чем за всю предыдущую историю человечества.
2. Второй тренд — взаимосвязанность данных. Информация перестала быть изолированной. Каждый кусочек знаний как-то связан с данными в других хранилищах информации. Страницы в интернете ссылаются на другие страницы. Тэги связывают помеченную информацию из разных источников. Онтологии устанавливают взаимосвязи между различными терминами, и т.д.
3. Третий тренд — использование слабоструктурированной информации. Возьмем простой пример: описание товара в магазине. Если раньше было достаточно 5-6 полей, чтобы описать одежду (размер, цвет, материал, фотография товара, …), то теперь количество параметров может доходить до нескольких десятков. Причем, для разных типов одежды будет использован разный набор параметров. В таких условиях становится крайне сложно заранее определить структуру таблицы, в которой хранится описание товара.
4. Четвертый тренд — архитектура. В 80-х годах прошлого века типичная архитектура использовала один большой компьютер (mainframe) и одну базу данных. В 90-х, распространение получила клинт-серверная архитектура. В наше время активно используются web-сервисы, каждый со своим backend-ом (со своей базой данных) и  распределенными решениями.

В таких условиях у реляционных баз данных резко падает производительность. И если для большинства web-сайтов производительности еще хватает, то для таких приложений как современные социальные сети или поисковые сервисы, SQL базы данных оказались несостоятельны.

Для большего понимания проведем сравнение реляционных и NoSQL решений:

- Стуруктуры данных и их типы - реляционные БД используют строгие схемы данных, NoSQL БД допускают любой тип данных.
- Запросы - вне зависимости от типа лицензии, реляционные базы данных в той или иной мере соответствуют стандартам SQL, поэтому данные из них можно получать при помощи языка SQL. NoSQL БД используют специфические способы запросов к данным.
- Масштабируемость - оба эти типа СУБД довольно легко поддаются вертикальному масштабированию (т.е. увеличение системных ресурсов). Тем не менее, так как NoSQL это более современный продукт, именно такие СУБД предлагают более простые способы горизонтального масштабирования (т.е. создание кластера из нескольких машин).
- Надежность - когда дело доходит до сохранности данных и гарантии выполнения транзакций реляционные БД по прежнему занимают лидирующие позиции.
- Поддержка - Реляционные СУБД имеют не малую историю за плечами. Они очень популярны и предлагают как платные, так и бесплатные решения. При возникновении проблем, все же гораздо проще найти ответ, если дело касается реляционных систем, чем NoSQL, особенно если решение довольно сложное по своей природе (например MongoDB).
- Хранение и доступ к данных - изначально реляционные системы предполагали быструю работу с данными, именно поэтому они превосходят остальные решения по производительности.

Как было показано на рисунке, существует два класса NoSQL систем: AP системы: CouchDB, Cassandra, Riak  и др.; CP системы: MongoDB, Redis, HBase и др.

Внутри каждого класса систем проводят также классификацию по принципам реализации хранилища данных.
Всего выделяют 4 типа: 

- 1 тип – хранилище по принципу ключ-значение; 
- 2 тип  - документно-ориентированное хранилище; 
- 3 тип - графовое хранилище; 
- 4 тип - колоночное хранилище.

**Хранилище по принципу ключ-значение**

Ключ может быть синтетическим или автосгенерированным, а значение может быть представлено строкой, JSON, блобом (BLOB, Binary Large Object) и т.д.

Такие базы данных как правило используют хеш-таблицу, в которой находится уникальный ключ и указатель на конкретный объект данных. Существует понятие блока (bucket) — логической группы ключей, которые не группируют данные физически. В разных блоках могут быть идентичные ключи.

Производительность сильно вырастает за счёт кеширующих механизмов, которые работают на основе маппингов. Чтобы прочитать значение, вам нужно знать как ключ, так и блок, поскольку на самом деле ключ является хешем (блок + ключ).

<img src="ris2.png" width=500 hight=500>

База данных такого типа позволяет читать и записывать значения с помощью ключа следующим образом:
- Get(key) возвращает значение, связанное с переданным ключом;
- Put(key, value) связывает значение с ключом;
- Multi-get(key1, key2, ..., keyN) возвращает список значений, связанных с переданным ключами;
- Delete(key) удаляет запись для ключа из хранилища.

И хотя базы данных типа «ключ-значение» могут пригодиться в определённых ситуациях, они не лишены недостатков. Первый заключается в том, что модель не предоставляет стандартные возможности баз данных вроде атомарности транзакций или согласованности данных при одновременном выполнении нескольких транзакций. Такие возможности должны предоставляться самим приложением.
Второй недостаток в том, что при увеличении объёмов данных, поддержание уникальных ключей может стать проблемой. Для её решения необходимо как-то усложнять процесс генерации строк, чтобы они оставались уникальными среди очень большого набора ключей.

Такие системы, как правило, относятся к классу AP систем.

Наиболее популярны в этой области: [Redis](https://ru.wikipedia.org/wiki/Redis), [Riak](https://ru.wikipedia.org/wiki/Riak), [Amazon DynamoDB](https://ru.wikipedia.org/wiki/DynamoDB).

**Документно-ориентированное хранилище** позволяет эффективно хранить и индексировать иерархические структуры данных. Такие системы в основном используются в документированных информационных системах.

В принципе здесь используется тот же самый подход ключ-значение, однако в качестве значений могут выступать структуры, которые также включают в себя наборы ключ-значение. Такие значения рассмотривают как документы. Поддерживаются различные форматы: XML, JSON и BSON — некоторые из наиболее распространенных.
В следующем примере можно увидеть данные в виде «документа» который отображает названия определённых магазинов. Обратите внимание, что, хотя все три примера содержат местоположение, они отображают его по-разному:

{officeName: "3Pillar Noida", 
   {Street: "B-25", City: "Noida",
   State: "UP", Pincode: "201301"}
}
{officeName: "3Pillar Timisoara",
   {Boulevard: "Coriolan Brediceanu No. 10",
   Block: "B, Ist Floor",
   City: "Timisoara", Pincode: "300011"}
}
{officeName: "3Pillar Cluj",
   {Latitude: "40.748328",
   Longitude: "-73.985560"}
}

Одним из ключевых различий между хранилищем «ключ-значение» и документоориентированным является то, что последний включает метаданные, связанные с хранимым содержимым, что даёт возможность делать запросы на основе содержимого. Например, в указанном примере можно попробовать найти все документы, в которых «City» равно «Noida», что вернёт все документы, связанные с магазинами в этом городе.

Как правило, такие хранилища относятся к CP системам.

Наиболее популярны в этой области: [CouchDB](https://ru.wikipedia.org/wiki/CouchDB), [MongoDB](https://ru.wikipedia.org/wiki/MongoDB)


**Графовые хранилища** наиболее подходят для хранения сложных бинарных отношений, возникающих между элементами данных информационной системы. 

Графовые структуры используются вместе с рёбрами, узлами и свойствами, что обеспечивает безиндексную смежность. При использовании графового хранилища данные могут быть легко преобразованы из одной модели в другую.

<img src="ris3.png" width=500 height=500>

Примерами графовых баз можно назвать: [Neo4j](https://ru.wikipedia.org/wiki/Neo4j), [Amazon Neptune](https://ru.wikipedia.org/wiki/Amazon_Neptune).

**Колоночные хранилища** представляют данные в виде некоторой разряженной матрицы, в которой строки и столбцы являются ключами с временными метками. Этот подход применяется для высоконагруженных систем, к которым предъявляется требование высокой скорости отклика. 

В колоночных NoSQL базах данных данные хранятся в ячейках, сгруппированных в колонки, а не в строки данных. Колонки логически группируются в колоночные семейства. Колоночные семейства могут состоять из практически неограниченного количества колонок, которые могут создаваться во время работы программы или во время определения схемы. Чтение и запись происходит с использованием колонок, а не строк.
В сравнении с хранением данных в строках, как в большинстве реляционных баз данных, преимущества хранения в колонках заключаются в быстром поиске/доступе и агрегации данных. Реляционные базы данных хранят каждую строку как непрерывную запись на диске. Разные строки хранятся в разных местах на диске, в то время как колоночные базы данных хранят все ячейки, относящиеся к колонке, как непрерывную запись, что делает операции поиска/доступа быстрее.

Пример: получение списка заголовков нескольких миллионов статей будет трудоёмкой задачей при использовании реляционных баз данных, так как для извлечения заголовков придётся проходить по каждой записи. А можно получить все заголовки с помощью только одной операции доступа к диску.

Двумерная таблица, состоящая из строк и колонок, является частью реляционной системы баз данных.

<img src="ris4.png" height=500 width=500>

Эту таблицу можно представить в виде BigTable-сопоставления следующим образом:

In [ ]:
{
   3PillarNoida: {
      city: Noida
      pincode: 201301
   },
   details: {
      strength: 250
      projects: 20
   }
}
{
   3PillarCluj: {
      address: {
         city: Cluj
         pincode: 400606
      }, 
      details: {
         strength: 200
         projects: 15
      }
   },
{
   3PillarTimisoara: {
      address: {
         city: Timisoara
         pincode: 300011
      },
      details: {
         strength: 150
         projects: 10
      }
   }
{
   3PillarFairfax : {
      address: {
         city: Fairfax
         pincode: VA 22033
      }, 
      details: {
         strength: 100
         projects: 5
      }
}

- Внешние ключи «3PillarNoida», «3PillarCluj», «3PillarTimisoara» и «3PillarFairfax» являются аналогами строк.
- «address» и «details» — колоночные семейства.
- В колоночном семействе «address» есть колонки «city» и «pincode».
- В колоночном семействе «details» есть колонки «strength» и «projects».
На колонки можно ссылаться с помощью колоночного семейства.

Самыми известными системами в этом классе являются: [BigTable](https://ru.wikipedia.org/wiki/BigTable), [Cassandra](https://ru.wikipedia.org/wiki/Apache_Cassandra), [HBase](https://ru.wikipedia.org/wiki/HBase).



## Распределенные хранилища данных. Технологии шардинга, репликации, MapReduce

При организации распределенного хранилища используется ряд специфических понятий: кластер, шардинг, репликация, технология MapReduce. 

При  вычислении  **кластер**  представляет  собой  тесно  связанную  совокупность  серверов  или  узлов.  Эти  серверы,  как правило,  имеют  одинаковую  аппаратную  спецификацию  и  соединены  вместе  через  сеть  для  работы  в  качестве  единого  блока.  Каждый  узел  в  кластере  имеет  свои  собственные  выделенные  ресурсы,  такие  как  память,  процессор  и  жесткий  диск.  Кластер  может  выполнять  задачу,  разбивая  ее  на  мелкие  кусочки  и  распределяя  их  выполнение  на  разных  компьютерах,  принадлежащих  кластеру.

**Файловая  система**  —  это  способ  хранения  и  организации  данных  на  запоминающих  устройствах,  таких  как  флэш-накопители,  DVD-диски  и  жесткие  диски.  Файл  —  это  неделимая  единица  хранения,  используемая файловой  системой  для  хранения  данных. 
Файловая  система  обеспечивает  логическое представление  данных,  хранящихся  на  запоминающем  устройстве,  и  представляет  его в  виде  древовидной  структуры  каталогов  и 
файлов.  Операционные  системы  используют  файловые  системы  для  хранения  и  считывания  данных 
по  поручению  приложений.  Каждая  операционная  система поддерживает  одну  или  несколько  файловых  систем,  например  NTFS  в  Microsoft  Windows  и  ext  в  Linux.

**Распределенная  файловая  система**  —  файловая  система,  которая  может  хранить  большие  файлы,  распределенные  по узлам  кластера.  Для  клиента  файлы  выглядят  как  локальные  объекты;  однако  это  только  логическое  представление, так  как  физически  файлы распределены  по  всему  кластеру.  Эти  локальные  отображаемые  элементы  представлены  благодаря  распределенной файловой  системе,  которая  позволяет  осуществлять  доступ к  файлам  из  разнообразных  мест.  Примерами  могут  быть  файловая  система  Google  ([GFS](https://ru.wikipedia.org/wiki/Google_File_System_))  и  распределенная  файловая система  Hadoop  ([HDFS](https://ru.wikipedia.org/wiki/Hadoop)).

**Шардинг**—  это  процесс  горизонтального  разделения  большого  массива  данных  на  коллекцию  меньших,  которые  являются  более  управляемыми  и  называются  **шардами**.  

Шарды  распределены  среди  множества  узлов,  где  узел  —  это  сервер или  компьютер.  

Каждый  сегмент  совместно  использует одну  и  ту  же  логическую  структуру  в  базе  данных,  и  все  вместе  шарды  представляют  собой  полный  набор  данных.

<img src="ris5.png" width=500 height=500>

Шардинг  позволяет  распределять  нагрузки  по  обработке  между  несколькими  узлами  для  достижения  горизонтальной  масштабируемости.

Горизонтальное  масштабирование  —  это  способ  увеличения  производительности  системы  за  счет  добавления  аналогичных  или  более  мощных  ресурсов  наряду  с  существующими  ресурсами.  Поскольку  каждый  узел  отвечает  только  за  часть  всего  массива  данных,  то  при  этом  время  чтения/записи 
значительно  улучшается.

Каждый  сегмент  может  независимо  обслуживать  операции  чтения  и  записи  для  определенного  подмножества данных,  за  которые  он  отвечает.

В  зависимости  от  запроса  данные  могут  быть  выбраны  из обоих  сегментов.

Преимуществом  шардинга  является  то,  что  оно  обеспечивает  частичную  отказоустойчивость  к  сбоям.  В  случае  сбоя узла,  только  данные  хранящиеся  на  этом  узле  подвержены влиянию.  Что  касается  разбиения  данных,  то  необходимо  учитывать  шаблоны  запросов,  чтобы  сами  сегменты  не  становились  узкими  местами  производительности. Например,  запросы,  требующие  данные  из  нескольких  сегментов,  накладывают  ограничение  на  производительность. 

Локальное  размещение  данных  позволяет  хранить  общедоступные  данные,  расположенные  в  одном  сегменте,  и  помогает  противостоять  таким  проблемам  с  производительностью.


**Репликация**  хранит  множество  копий  наборов  данных,  известных  как  «реплики»,  на  нескольких  узлах.

Репликация  обеспечивает  масштабируемость  и  доступность  благодаря  тому  факту,  что  одни  и  те  же  данные  реплицируются  на  разных  узлах.  При  этом  также  достигается  отказоустойчивость,  поскольку  избыточность  данных  гарантирует, что  данные  не  будут  потеряны  в  случае,  когда  отдельный 
узел  выходит  из  строя.  

Для  реализации  репликации  используются  два  разных  метода:
- ведущий-ведомый
- равный  к  равному  (пиринговый)

Во  время  однонаправленного  режима  репликации  «**ведущий-ведомый**»  узлы  размещаются  в  конфигурации  ведущий-ведомый,  и  все  данные  записываются  в  главный  узел. 

После  сохранения  данные  реплицируются  на  несколько  подчиненных  узлов.  Все  внешние  запросы  на  запись,  включая вставку,  обновление  и  удаление  данных,  происходят  на  главном  узле,  в  то  время,  как  запросы  на  чтение  могут  выполняться  любым  подчиненным  узлом.

Если  ведущий  узел  выйдет  из  строя,  то  чтение  по-прежнему  будет  возможным  через  любой  из  ведомых узлов.

Ведомый  узел  может  быть  сконфигурирован  как  резервный узел  для  ведущего  узла.  И  в  случае  сбоя  основного  узла  запись  не  будет  поддерживаться  до  тех  пор,  пока  ведущий  узел 
не  будет  восстановлен.
 
Ведущий  узел  восстанавливается  либо  из  резервной  копии  ведущего  узла,  либо  из  подчиненных  узлов  выбирается  новый  ведущий  узел.  Одна  из  проблем,  связанных  с  однонаправленным  режимом  репликации  «ведущий-ведомый»,— это  несогласованность  чтения,  что  может  быть  проблемой,  если  ведомый  узел  читается  до  обновления  ведущего,  который  его  копирует.
 
Для  обеспечения  согласованности  чтения,  там  может  быть  реализована  система  голосования  в  том  случае,  если  чтение объявлено  согласованным,  а  большинство  ведомых  узлов содержат  одну  и  ту  же  версию  записи.  Для  внедрения  такой  системы  голосования  требуется  надежный  и  быстрый  механизм  обмена  данными  между  ведомыми  узлами.

При  репликации  в  режиме  «**одноранговый**»  все  узлы  работают  на  одном  уровне.  Другими  словами,  между  узлами  нет  отношения  «ведущий-ведомый».  Каждый  узел,  известный  как  одноранговый  узел,  в  равной  степени  способен  обрабатывать операции  чтения  и  записи.  Каждая  запись  копируется  во  все 
одноранговые  узлы.

Одноранговая  репликация  подвержена  ошибкам  записи,  возникающим  в  результате  одновременного  обновления  одних тех  же  данных  несколькими  одноранговыми  узлами.  Эту  проблему  можно  решить  путем  реализации  либо  пессимистической,  либо  оптимистической  стратегии  распараллеливания.

- Пессимистический  параллелизм  представляет  собой  упреждающую  стратегию,  которая  предотвращает  несогласованность.  Она  использует  блокировки  для  гарантии  того,  что  только  одно  обновление  на  запись  может  происходить  одновременно.  Однако  это  наносит  ущерб  доступности,  поскольку  обновляемая  запись  базы  данных  остается  недоступной  до  тех  пор,  пока  не  будут  освобождены  все  блокировки.
- Оптимистичный  параллелизм  —  это  реактивная  стратегия,  которая  не  использует  блокировки.  Вместо  этого  она допускает  несогласованность,  предполагая,  что  в  конечном  итоге  согласованность  будет  достигнута  после  того, как  все  обновления  распространятся.

При  оптимистичном  параллелизме  одноранговые  узлы  могут  оставаться  несогласованными  в  течение периода  времени,  прежде  чем  достигнут  согласованности.  
Однако  база  данных  остается  доступной,  поскольку  блокировка не  задействована.  Подобно  режиму  однонаправленной  репликации  «ведущий-ведомый»,  чтение  может  быть  несогласованным  в  течение  периода  времени,  когда  некоторые  из одноранговых  узлов  завершили  свои  обновления,  в  то  время как  другие  еще  выполняют  свои  обновления.  Тем  не  менее, чтение  в  конечном  итоге  становится  согласованным,  когда обновления  были  выполнены  для  всех одноранговых узлов.

Для  обеспечения  согласованности  чтения,  там  может  быть реализована  система  голосования  в  том  случае,  если  чтение  объявлено  согласованным,  а  большинство  одноранговых  узлов  содержат  одну  и  ту  же  версию  записи.  Как  отмечалось ранее,  внедрение  такой  системы  голосования  требует  надежного  и  быстрого  механизма  обмена  данными  между  одноранговыми  узлами.

Для  улучшения  ограниченной  отказоустойчивости,  обеспечиваемую  с  помощью  использования  приемов  шардинга,  а  также  для  получения  дополнительных  преимуществ  от повышенной  доступности  и масштабируемости  репликации,  можно  комбинировать как шардинг,  так  и  репликацию.


Возможности обработки запросов NoSQL базы данных напрямую следуют из ее модели распределения. 

Популярное решения для распределенных вычислений кластеров NoSQL базы данных – **MapReduce**. 

MapReduce работает в два шага:
1. Map-шаг: предварительная обработка полученных данных. На этом шаге Мастер или один из Мастер-серверов получает данные задачи, разделяет их на части и передает для предварительной обработки другим узлам.
2. Reduce-шаг: свертка обработанных данных. Мастер получает от прочих узлов подготовленные данные, формирует и отдает ответ, который является решением входной задачи.

## OLAP-технологии

У истоков технологии OLAP стоит основоположник реляционного подхода Э. Кодд. В 1993 г. он опубликовал статью под названием "OLAP для пользователей-аналитиков: каким он должен быть".

Кодд рассмотрел ее недостатки реляционной модели БД, указав в первую очередь на невозможность "объединять, просматривать и анализировать данные с точки зрения множественности измерений, т. е. самым понятным для аналитиков способом".

Далее он  определяет необходимость разработки некоторого расширенного варианта реляционной модели, необходимого для оперативной аналитической обработки данных. Для этого вводится ряд понятий.

**Измерение (Dimensions)** - это последовательность значений одного из анализируемых параметров. Например, для параметра "время" это последовательность календарных дней, для параметра "муниципальное образование" это, например, список городов.

Множественность измерений предполагает представление данных в виде многомерной модели. По измерениям в многомерной модели откладывают параметры, относящиеся к анализируемой предметной области.

По Кодду, многомерное концептуальное представление (multi-dimensional conceptual view) — это множественная перспектива, состоящая из нескольких независимых измерений, вдоль которых могут быть проанализированы определенные совокупности данных. Одновременный анализ по нескольким измерениям определяется как многомерный анализ.

Каждое измерение может быть представлено в виде иерархической структуры. Например, измерение "Исполнитель" может иметь следующие иерархические уровни: "предприятие - подразделение - отдел - служащий". Более того, некоторые измерения могут иметь несколько видов иерархического представления. Например, измерение "время" может включать две иерархии со следующими уровнями: "год - квартал -  месяц - день" и "неделя — день". 

На пересечениях осей измерений (Dimensions) располагаются данные, количественно характеризующие анализируемые факты, **меры (Measures)**. Это могут быть объемы продаж, выраженные в единицах продукции или в денежном выражении, остатки на складе, издержки и т. п.

<img src="ris6.png" width=300 height=300>

Таким образом, многомерную модель данных можно представить как гиперкуб.

Над таким гиперкубом могут выполняться следующие операции:

**Срез (Slice)** — формируется подмножество многомерного массива данных, соответствующее единственному значению одного или нескольких элементов измерений. 

Если рассматривать термин "срез" с позиции конечного пользователя, то наиболее часто его роль играет двумерная проекция куба.

<img src="ris7.png" width=400 height=400>

**Вращение (Rotate)**  - изменение расположения измерений, представленных в отчете или на отображаемой странице. Например, операция вращения может заключаться в перестановке местами строк и столбцов таблицы или перемещении интересующих измерений в столбцы или строки создаваемого отчета, что позволяет придавать ему желаемый вид. 

<img src="ris8.png" width=400 height=400>

**Консолидация (Drill Up) и детализация (Drill Down)**  — операции, которые определяют переход вверх по направлению от детального (down) представления данных к агрегированному (up) и наоборот, соответственно. Направление детализации (обобщения) может быть задано как по иерархии отдельных измерений, так и согласно прочим отношениям, установленным в рамках измерений или между измерениями.

<img src="ris9.png" width=400 height=400>

С концепцией многомерного анализа данных тесно связывают оперативный анализ, который выполняется средствами OLAP-систем.

**OLAP (On-Line Analytical Processing)** — технология оперативной аналитической обработки данных, использующая методы и средства для сбора, хранения и анализа многомерных данных в целях поддержки процессов принятия решений.

В данной статье Коддом были сформулированы 12 принципов:

Ниже перечислены 12 правил, изложенных Коддом и определяющих OLAP.

1. **Многомерность** -  OLAP-система на концептуальном уровне должна представлять данные в виде многомерной модели, что упрощает процессы анализа и восприятия информации.

2. **Прозрачность** - OLAP-система должна скрывать от пользователя реальную реализацию многомерной модели, способ организации, источники, средства обработки и хранения.

3. **Доступность** - OLAP-система должна предоставлять пользователю единую, согласованную и целостную модель данных, обеспечивая доступ к данным независимо от того, как и где они хранятся.

4. **Постоянная производительность при разработке отчетов** -  производительность OLAP-систем не должна значительно уменьшаться при увеличении количества измерений, по которым выполняется анализ.

5. **Клиент-серверная архитектура** - OLAP-система должна быть способна работать в среде "клиент-сервер", т. к. большинство данных, которые сегодня требуется подвергать оперативной аналитической обработке, хранятся распределенно. Главной идеей здесь является то, что серверный компонент инструмента OLAP должен быть достаточно интеллектуальным и позволять строить общую концептуальную схему на основе обобщения и консолидации различных логических и физических схем корпоративных БД для обеспечения эффекта прозрачности.

6. **Равноправие измерений** — OLAP-система должна поддерживать многомерную модель, в которой все измерения равноправны. При необходимости дополнительные характеристики могут быть предоставлены отдельным измерениям, но такая возможность должна быть предоставлена любому измерению.

7. **Динамическое управление разреженными матрицами** -  OLAP-система должна обеспечивать оптимальную обработку разреженных матриц. Скорость доступа должна сохраняться вне зависимости от расположения ячеек данных и быть постоянной величиной для моделей, имеющих разное число измерений и различную степень разреженности данных.

8. **Поддержка многопользовательского режима** -  OLAP-система должна предоставлять возможность работать нескольким пользователям совместно с одной аналитической моделью или создавать для них различные модели из единых данных. При этом возможны как чтение, так и запись данных, поэтому система должна обеспечивать их целостность и безопасность.

9. **Неограниченные перекрестные операции** -  OLAP-система должна обеспечивать сохранение функциональных отношений, описанных с помощью определенного формального языка между ячейками гиперкуба при выполнении любых операций среза, вращения, консолидации или детализации. Система должна самостоятельно (автоматически) выполнять преобразование установленных отношений, не требуя от пользователя их переопределения.

10. **Интуитивная манипуляция данными** - OLAP-система должна предоставлять способ выполнения операций среза, вращения, консолидации и детализации над гиперкубом без необходимости пользователю совершать множество действий с интерфейсом. Измерения, определенные в аналитической модели, должны содержать всю необходимую информацию для выполнения вышеуказанных операций.

11. **Гибкие возможности получения отчетов** -  OLAP-система должна поддерживать различные способы визуализации данных, т. е. отчеты должны представляться в любой возможной ориентации. Средства формирования отчетов должны представлять синтезируемые данные или информацию, следующую из модели данных в ее любой возможной ориентации. Это означает, что строки, столбцы или страницы должны показывать одновременно от 0 до N измерений, где N число измерений всей аналитической модели. Кроме того, каждое измерение содержимого, показанное в одной записи, колонке или странице, должно позволять показывать любое подмножество элементов (значений), содержащихся в измерении, в любом порядке.

12. **Неограниченная размерность и число уровней агрегации** -  исследование о возможном числе необходимых измерений, требующихся в аналитической модели, показало, что одновременно может использоваться до 19 измерений. Отсюда вытекает настоятельная рекомендация, чтобы аналитический инструмент мог одновременно предоставить хотя бы 15, а предпочтительно- 20 измерений. Более того, каждое из общих измерений не должно быть ограничено по числу определяемых пользователем-аналитиком уровней агрегации и путей консолидации.


Основным достоинством многомерной модели данных является ее нацеленность на сложные процедуры аналитической обработки, для которых использование обычной реляционной модели приводит к нелинейному росту трудоемкости, существенному возрастанию затрат памяти на индексацию данных. Основным недостатком становится громозкость и сложность этой модели для простейших операций обработки данных. Примерами систем, поддерживающих многомерные модели данных, являются [Essbase (Arbor Software)](https://ru.wikipedia.org/wiki/Essbase),
[Microsoft SQL Server Analysis Services](https://ru.wikipedia.org/wiki/Microsoft_Analysis_Services),  [Oracle Database OLAP Option](https://en.wikipedia.org/wiki/Oracle_OLAP), [IBM Cognos TM1](https://ru.wikipedia.org/wiki/TM1); существует несколько свободных решений, среди них отмечаются [Mondrian](https://ru.wikipedia.org/wiki/Mondrian_(%D1%81%D0%B5%D1%80%D0%B2%D0%B5%D1%80_OLAP)) и [Palo](https://en.wikipedia.org/wiki/Palo_(OLAP_database))


Еще одним интересным подходом стала идея распространения принципов объектно-ориентированного программирования на область работы с данными. Основная идея этого класса методов состоит в отказе от табличной формы представления данных, с возможностью идентифицировать отдельные записи базы данных. Также за счет использования принципов наследования реализуется идея вложенных связей, аналогичных связям в реляционной модели: один к одному, один ко многим, многие ко многим. За счет использования полиморфизма появляется возможность унификации процедур работы с данными с использованием функций с одним и тем же именем, но меняющим свой функционал в зависимости от типа данных. За счет принципа инкапсуляции  появляется возможность ввести сложные ограничения на доступ к данным. В полном объеме стандартизованная объектно-ориентированной модель описана в рекомендациях стандарта ODMG-93 (Object Database Management Group — группа управления объектно-ориентированными базами данных). 

Укажем также на необходимость учета темпоральности – изменчивости данных. Это приводит к еще одной проблеме, которая до сих пор не нашла в рамках реляционной модели полного решения. Действительно, традиционные реляционные БД, как правило, хранят информацию лишь о текущем состоянии. СУБД могут работать с темпоральными данными (определённые даты и интервалы времени). Однако общепризнанного подхода до сих пор не разработано, а учет темпоральности не реализован полностью ни в одной реализации крупных коммерческих СУБД. В тоже время во многих областях учет темпоральности  - важная задача. Например в таких областях как: банковское дело, бухгалтерский учёт, т.п. 
В темпоральных базах данных целесообразно учитывать следующие типы моментов времени: модельное время (определяет ход глобального времени для всей системы), время фиксации факта (определяет факт возникновения некоторого события), время операции (определяет время, прошедшее между последовательными изменениями одного и того же атрибута данных).

Еще одним из интересных направлений является модель данных Сущность – Атрибут – Значение (EAV)

Модель данных сущность – атрибут – значение EAV (Entity-Atribute-Value) является важным подходом в направлении поиска универсальной структуры данных, не накладывающей таких жестких ограничений, как реляционная модель. По сути  - это еще один из альтернативных подходов, устраняющий проблемы статичности структуры реляционных данных. Данный подход часто также называют «вертикальной моделью данных», в противовес реляционной модели, где атрибуты располагаются горизонтально.

Полноценно данный подход стал рассматриваться в 1999 году после работ Прадкаша Надкари. П. Надкари исследовал возможности внедрения реляционной модели в предметной области медицины и впервые столкнулся с ограничениями. Основная проблема состояла в фиксации в информационной системе наличия или отсутствия симптомов какого-либо заболевания. 

Особенность состояла в том, что если каждый из симптомов обозначать атрибутом реляционной таблицы, то большинство атрибутов останутся незаполненными (со значение NULL), поскольку далеко не все симптомы интересны для той или иной болезни. Все это быстро вырождало хранилище биомедицинских данных в неэффективную информационную систему. 

В дальнейшем оказалось, что наиболее широко используемая реляционная модель данных неэффективна  в качестве среды «быстрой» разработки приложений либо создании базы данных для хранения данных с динамически изменяющейся структурой. В обоих данных случаях имеется  набор сущностей и их атрибутов, который заранее неизвестен и потенциально велик, но фактическое их количество, используемое в базе данных, сравнительно мало. 

Важно отметить, что в реляционной модели логическая модель данных повторяет физическую модель, поэтому любые изменения логической модели в процессе жизненного цикла информационной системы отражаются на физической модели и приводят к необходимости переработки программного кода с существенными затратами.

В EAV модели логическая модель универсальна и не претерпевает изменения в ходе жизненного цикла информационной системы. Это позволяет добиваться существенной гибкости как раз в случаях изменения динамических связей между объектами базы данных.

Набор основных понятий для данной модели следующий:

- «Сущность» - содержит уникальный идентификатор сущности. Также может представлять собой пару (идентификатор, временная метка) для отражения хронологии событий и фактов;
- «Атрибут» - название свойства сущности, как правило — ссылка (короткий идентификатор) на таблицу атрибутов в метаданных;
- «Значение» - непосредственно значение атрибута.
Упрощенная классическая модель EAV представлена на рисунке:

<img src="ris10.png" width=400 height=400>

Как видно из рисунка, модель в общем виде включает в себя только три таблицы: таблица сущности  - количество сущностей равно количеству таблиц реляционной модели; таблица атрибуты – включает в себя ссылку на таблицу сущности; а также название атрибута этой сущности; таблица значение  - содержит значение атрибута у заданного объекта заданной сущности.
Данную EAV модель легко транслировать в виде ER модели, на рисунке представлена общая схема такой ER модели.

<img src="ris11.png" width=400 height=400>
